# Alligators Don't Apologize - Exploratory Data Analysis

In [8]:
import pandas as pd

In [9]:
# Path to the data folders
raw_data_folder = '../data/RAW/'
clean_data_folder = '../data/CLEAN/'

## First Dataset: CMU Movies

In [10]:
# Load the movie metadata
CMU_movie_metadata = pd.read_csv(raw_data_folder+'CMU_Movies_Dataset/movie.metadata.tsv', delimiter='\t', header=None)
CMU_movie_metadata.columns = ['wikipedia_movie_ID', 'freebase_movie_ID', 'movie_name', 'movie_release_date', 'movie_box_office_revenue', 'movie_runtime', 'movie_languages', 'movie_countries', 'movie_genres']
CMU_movie_metadata.head(10)

# Load the character metadata
CMU_char_metadata = pd.read_csv(raw_data_folder+'CMU_Movies_Dataset/character.metadata.tsv', sep='\t', header=None)
CMU_char_metadata.columns = ['wikipedia_movie_ID', 'freebase_movie_ID', 'movie_release_date', 'character_name', 'actor_DOB', 'actor_gender', 'actor_height_meters', 'actor_ethnicity', 'actor_name', 'actor_age_at_movie_release', 'freebase_character_actor_map_ID', 'freebase_character_ID', 'freebase_actor_ID']

# Load the plot summaries
CMU_plot_summary = pd.read_csv(raw_data_folder+'CMU_Movies_Dataset/plot_summaries.txt', sep='\t', header=None)
CMU_plot_summary.columns = ['wikipedia_movie_ID', 'plot_summary']

#######

# Include the other parts of the dataset if needed

######


## Second Dataset: GVD Dataset

In [11]:
# Load the GVD dataset
GVD_data = pd.read_csv(raw_data_folder+'GVD_Dataset/2023_gvdDatabase_1_0_country.csv')

In [12]:
GVD_data.head(10)

,year,country_code,country_name,country_region,country_subregion,population,indicator,mechanism,count,rate,count_population,source_name,source_url,entry_comment
0,2004,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,0,0.00,51716,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2009 up...
1,2004,ABW,Aruba,Americas,Caribbean,Male,Homicide intentional,All,2,4.25,47028,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2009 up...
2,2004,ABW,Aruba,Americas,Caribbean,Total,Homicide intentional,All,2,2.03,98744,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2009 up...
3,2005,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,1,1.91,52454,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2010 up...
4,2005,ABW,Aruba,Americas,Caribbean,Male,Homicide intentional,All,5,10.51,47574,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2010 up...
5,2005,ABW,Aruba,Americas,Caribbean,Total,Homicide intentional,All,6,6.00,100028,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2010 up...
6,2006,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,1,1.89,52895,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2011 up...
7,2006,ABW,Aruba,Americas,Caribbean,Male,Homicide intentional,All,4,8.34,47935,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2011 up...
8,2006,ABW,Aruba,Americas,Caribbean,Total,Homicide intentional,All,5,4.96,100830,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2011 up...
9,2007,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,0,0.00,53086,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2012 up...


## Third dataset: Kaggle Movies Dataset

In [13]:
# Load the movies metadata
Kaggle_movies_metadata = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/movies_metadata.csv')

# Load the credits
Kaggle_credits = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/credits.csv')

# Load the keywords data
Kaggle_keywords = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/keywords.csv')

# Load the ratings
Kaggle_ratings = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/ratings.csv')

/var/folders/8z/vm21_w1s46v5nvgv_mq9wlrh0000gn/T/ipykernel_41113/680486532.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  Kaggle_movies_metadata = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/movies_metadata.csv')


In [14]:
Kaggle_movies_metadata.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
